# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [53]:
import pandas as pd
from numpy import random 
from citipy import citipy
import requests
from pprint import pprint
from config import weather_api_key
import json

## Generate Cities List

In [54]:
rand_lat_gen = random.uniform(-90,91,526)
rand_lon_gen = random.uniform(-180,181,526)

city_corr = {"rand_lat": rand_lat_gen,
            "rand_lon": rand_lon_gen}

city_corr = pd.DataFrame(city_corr)

city_list = []
country_list = []

for i in city_corr.index:
    latitude = city_corr.iloc[int(i),0]
    longitude = city_corr.iloc[int(i),1]
    location = citipy.nearest_city(latitude,longitude)
    city = location.city_name
    ctry = location.country_code
    city_list.append(city)
    country_list.append(ctry)
    
city_corr['City'] = city_list
city_corr['Country'] = country_list
city_corr.head()



,rand_lat,rand_lon,City,Country
0,1.166375,27.980388,wamba,cd
1,78.926383,-68.877668,qaanaaq,gl
2,31.932854,74.389180,narang,pk
3,45.494347,-89.888008,merrill,us
4,-84.074736,174.557537,bluff,nz


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [55]:
city_lats = []
city_lons = []
city_nums = []
city_names = []
cloudss = []
countrys = []
dates = []
humiditys = []
lats = []
lngs = []
max_temps = []
wind_speeds = []

for i in city_corr.index:
    city_name = city_corr.iloc[int(i),2]
    country_code = city_corr.iloc[int(i),3]
    
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name},{country_code}&appid={weather_api_key}&cnt=1&units=imperial"
    cities = requests.get(url).json()
    
    try:
        city_lat = cities['coord']['lat']
        city_lon = cities['coord']['lon']
        city_num = cities['id']
        city_name = cities['name']
        clouds = cities['clouds']['all']
        country = cities['sys']['country']
        date = cities['dt']
        humidity = cities['main']['humidity']
        lat = cities['coord']['lat']
        lng = cities['coord']['lon']
        max_temp = cities['main']['temp_max']
        wind_speed = cities['wind']['speed']
    except KeyError as e:
        print(e)
    
    city_lats.append(city_lat)
    city_lons.append(city_lon)
    city_nums.append(city_num)
    city_names.append(city_name)
    cloudss.append(clouds)
    countrys.append(country)
    dates.append(date)
    humiditys.append(humidity)
    lats.append(lat)
    lngs.append(lng)
    max_temps.append(max_temp)
    wind_speeds.append(wind_speed)

    print(f"Data found: {city_num} -- {city_name}, {country}")
    
city_corr['City_latitude'] =  city_lats
city_corr['City_longitude'] =  city_lons
city_corr['City ID'] =  city_nums
city_corr['City'] =  city_names
city_corr['Country'] =  countrys
city_corr['Cloudiness'] =  cloudss
city_corr['Date'] =  dates
city_corr['Humidity'] =  humiditys
city_corr['Lat'] =  lats
city_corr['Lng'] =  lngs
city_corr['Max Temp'] =  max_temps
city_corr['Wind Speed'] =  wind_speeds
    
city_corr = city_corr.drop_duplicates(subset=['City','Country'], keep='first', inplace=False)

Data found: 204318 -- Wamba, CD
Data found: 3831208 -- Qaanaaq, GL
Data found: 1169334 -- Narang Mandi, PK
Data found: 5262680 -- Merrill, US
Data found: 2206939 -- Bluff, NZ
Data found: 3443061 -- Chui, UY
Data found: 2303611 -- Esim, GH
Data found: 3424607 -- Tasiilaq, GL
Data found: 3896218 -- Castro, CL
Data found: 1631905 -- Payakumbuh, ID
Data found: 3361934 -- Saldanha, ZA
Data found: 3833367 -- Ushuaia, AR
Data found: 2075265 -- Busselton, AU
'coord'
Data found: 2075265 -- attawapiskat, AU
Data found: 3374333 -- Praia, CV
Data found: 3369157 -- Cape Town, ZA
'coord'
Data found: 3369157 -- maridi, ZA
Data found: 3430863 -- Mar del Plata, AR
Data found: 2163355 -- Hobart, AU
Data found: 2151187 -- Roma, AU
Data found: 3901501 -- Villazón, BO
Data found: 3369157 -- Cape Town, ZA
'coord'
Data found: 3369157 -- barentsburg, ZA
Data found: 4034551 -- Faanui, PF
Data found: 1706466 -- Ligayan, PH
Data found: 4030556 -- Rikitea, PF
Data found: 2121385 -- Severo-Kuril'sk, RU
Data found:

Data found: 4032243 -- Vaini, TO
Data found: 5880054 -- Barrow, US
Data found: 2527087 -- Tiznit Province, MA
Data found: 3652764 -- Puerto Ayora, EC
Data found: 3833367 -- Ushuaia, AR
Data found: 2112444 -- Kamaishi, JP
Data found: 144794 -- Ābyek, IR
Data found: 3874787 -- Punta Arenas, CL
Data found: 4903730 -- Niles, US
Data found: 6165406 -- Thompson, CA
Data found: 1033356 -- Nampula, MZ
Data found: 4993659 -- Garden City, US
Data found: 3833367 -- Ushuaia, AR
Data found: 3833367 -- Ushuaia, AR
'coord'
Data found: 3833367 -- tabiauea, AR
Data found: 5887916 -- Athabasca, CA
Data found: 4032243 -- Vaini, TO
Data found: 3370903 -- Jamestown, SH
Data found: 3418910 -- Upernavik, GL
Data found: 3652764 -- Puerto Ayora, EC
'coord'
Data found: 3652764 -- taolanaro, EC
Data found: 3421719 -- Narsaq, GL
Data found: 2411397 -- Georgetown, SH
Data found: 3029029 -- Arrondissement de Cambrai, FR
Data found: 3874787 -- Punta Arenas, CL
Data found: 4030556 -- Rikitea, PF
Data found: 1006984 -

Data found: 6148373 -- Sioux Lookout, CA
Data found: 5866583 -- Saint Paul Harbor, US
Data found: 80509 -- Al Bardīyah, LY
Data found: 5848280 -- Kapaa, US
Data found: 4011743 -- Constitucion, MX
Data found: 3933104 -- Pangoa, PE
Data found: 3355672 -- Lüderitz, NA
Data found: 1625908 -- Sungai Raya, ID
Data found: 1024312 -- Mocuba, MZ
Data found: 935215 -- Saint-Philippe, RE
Data found: 3927758 -- Acobamba, PE
'coord'
Data found: 3927758 -- meyungs, PE
Data found: 3587498 -- Acajutla, SV
Data found: 4032243 -- Vaini, TO
Data found: 3833367 -- Ushuaia, AR
Data found: 4030556 -- Rikitea, PF
'coord'
Data found: 4030556 -- amderma, PF
'coord'
Data found: 4030556 -- nioro, PF
'coord'
Data found: 4030556 -- belushya guba, PF
Data found: 2063042 -- Port Hedland, AU
Data found: 933995 -- Souillac, MU
Data found: 5380437 -- Pacific Grove, US
Data found: 3896218 -- Castro, CL
Data found: 2450173 -- Taoudenni, ML
Data found: 1058381 -- Morondava, MG
'coord'
Data found: 1058381 -- taolanaro, MG


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [56]:
city_corr.to_csv("output_data/city_weather.csv", index=False, header=True)
city_corr.head()

,rand_lat,rand_lon,City,Country,City_latitude,City_longitude,City ID,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,1.166375,27.980388,Wamba,CD,2.1484,27.9947,204318,92,1621893554,93,2.1484,27.9947,64.42,2.37
1,78.926383,-68.877668,Qaanaaq,GL,77.4840,-69.3632,3831208,5,1621893120,75,77.4840,-69.3632,32.52,6.62
2,31.932854,74.389180,Narang Mandi,PK,31.9132,74.5176,1169334,0,1621893554,29,31.9132,74.5176,82.36,5.30
3,45.494347,-89.888008,Merrill,US,45.1805,-89.6835,5262680,75,1621893336,69,45.1805,-89.6835,79.03,11.50
4,-84.074736,174.557537,Bluff,NZ,-46.6000,168.3333,2206939,6,1621893113,79,-46.6000,168.3333,39.29,7.09


In [57]:
# qurl = f"http://api.openweathermap.org/data/2.5/weather?q=palabuhanratu,id&appid={weather_api_key}&cnt=1&units=imperial"
# requests.get(qurl).json()


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [58]:
#  Get the indices of cities that have humidity over 100%.


In [59]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression